# Feature Generator 

데이터 EDA를 기반으로 새로운 feature을 생성하겠습니다.  
순서는 다음과 같습니다. 
1. 변수 생성: `누적 행동 데이터`
2. 변수 생성: `weekday`, `holiday`
3. 변수 생성: `hour`
4. 변수 생성: `latest`
5. 변수 생성: `prefer_dvc`
6. 변수 생성: `trfc_dvc`
7. 외부데이터: `온도`, `습도`, `강수량`, `강수형태`
<br><br>

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tq
pd.set_option('display.max_row',300)
%matplotlib inline
warnings.filterwarnings(action='ignore')

## load data

In [7]:
cks_dtype = {'clnt_id':'int64','sess_id':'int64','hit_seq':'int64','action_type':'int64','biz_unit':'category','sess_dt':'object','hit_tm':'object',
             'hit_pss_tm':'int64','trans_id':'float64','sech_kwd':'object','tot_pag_view_ct':'float64','tot_sess_hr_v':'float64','trfc_src':'category',
             'dvc_ctg_nm':'object','pd_c':'object','de_dt':'object','de_tm':'object','buy_am':'int64','buy_ct':'int64','clnt_gender':'category'}

online = pd.read_csv('./data/online_03.csv', dtype=cks_dtype)
trade = pd.read_csv('./data/trade_01.csv', dtype=cks_dtype)
customer = pd.read_csv('./data/customer_01.csv', dtype=cks_dtype)
item = pd.read_csv('./data/item_01.csv', dtype=cks_dtype)

In [8]:
online.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
0,1,1,1,0,A01,20190911,16:14,11880,NaN,과일선물세트,14.0,124.0,DIRECT,mobile_app
1,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,14.0,124.0,PUSH,mobile_app
2,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,14.0,124.0,DIRECT,mobile_app
3,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,45.0,424.0,DIRECT,mobile_app
4,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,45.0,424.0,DIRECT,mobile_app


In [9]:
df1_online = online.copy()

<br>

### 1. 변수 생성: `누적 행동 데이터`
- action_type을 활용하여 누적 행동 정보를 생성하겠습니다. 
- 누적 행동 정보는 고객의 과거 행동 추이를 확인 할 수 있습니다. 

In [10]:
df1_dummy = pd.get_dummies(df1_online['action_type'], prefix='cum_act', prefix_sep='_')
df1_online = pd.concat([df1_online, df1_dummy], axis=1)

In [15]:
df1_online

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,trfc_src,dvc_ctg_nm,cum_act_0,cum_act_1,cum_act_2,cum_act_3,cum_act_4,cum_act_5,cum_act_6,cum_act_7
0,1,1,1,0,A01,20190911,16:14,11880,NaN,과일선물세트,...,DIRECT,mobile_app,1,0,0,0,0,0,0,0
1,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,...,PUSH,mobile_app,2,0,0,0,0,0,0,0
2,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,...,DIRECT,mobile_app,3,0,0,0,0,0,0,0
3,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,...,DIRECT,mobile_app,4,0,0,0,0,0,0,0
4,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,...,DIRECT,mobile_app,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,72426,1,6,2,A03,20190930,00:21,282807,NaN,NaN,...,DIRECT,mobile_web,2,1,3,0,0,0,0,0
3196358,72426,1,7,0,A03,20190930,00:22,312037,NaN,핫도그,...,DIRECT,mobile_web,3,1,3,0,0,0,0,0
3196359,72426,1,8,1,A03,20190930,00:22,333968,NaN,NaN,...,DIRECT,mobile_web,3,2,3,0,0,0,0,0
3196360,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,...,DIRECT,mobile_web,0,1,0,0,0,0,0,0


In [11]:
# 각 행동 유형별로 누적해서 합해주는 함수.
for i in range(7):
    col = 'cum_act_'+str(i)
    df1_online[col] = df1_online.groupby('clnt_id')[col].cumsum() 

In [24]:
df1_online.head(205).tail(10)

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,trfc_src,dvc_ctg_nm,cum_act_0,cum_act_1,cum_act_2,cum_act_3,cum_act_4,cum_act_5,cum_act_6,cum_act_7
195,29,16,2,0,A01,20190929,17:10,93307,NaN,아이라인,...,DIRECT,mobile_app,56,0,0,0,0,0,1,0
196,29,16,3,0,A01,20190929,17:11,131229,NaN,쉐이딩,...,PUSH,mobile_app,57,0,0,0,0,0,1,0
197,30,1,1,0,A01,20190823,10:10,306650,NaN,제휴혜택,...,DIRECT,mobile_app,1,0,0,0,0,0,0,0
198,31,1,1,0,A01,20190808,15:24,0,NaN,전동킥보드악세사리,...,PORTAL_1,mobile_web,1,0,0,0,0,0,0,0
199,33,1,1,0,A01,20190807,17:53,8308,NaN,루이까또즈지갑,...,DIRECT,mobile_web,1,0,0,0,0,0,0,0
200,34,1,1,0,A01,20190703,22:14,22622,NaN,등산화,...,PORTAL_1,mobile_web,1,0,0,0,0,0,0,0
201,34,1,2,0,A01,20190703,22:14,49689,NaN,등산화,...,PORTAL_1,mobile_web,2,0,0,0,0,0,0,0
202,34,1,3,0,A01,20190703,22:15,76754,NaN,등산화,...,PORTAL_1,mobile_web,3,0,0,0,0,0,0,0
203,34,1,4,0,A01,20190703,22:16,148359,NaN,등산화,...,PORTAL_1,mobile_web,4,0,0,0,0,0,0,0
204,34,1,5,0,A01,20190703,22:17,215510,NaN,등산화,...,PORTAL_1,mobile_web,5,0,0,0,0,0,0,0


<br>

### 2. 변수 생성: `weekday`, `holiday`
- 날짜 데이터를 활용하여 `weekday` 요일을 생성하겠습니다. 
- (예) 요일숫자(0-월, 1-화, 2-수, ....)
<br>

- 주말, 광복절, 추석은 `holiday`로 생성합니다. 
- (예) '2019-08-15'-광복절, '2019-09-12'-추석연휴,'2019-09-13'-추석연휴
    - 주말과 공휴일은 1, 그 외 요일은 0이 됩니다. 
    - hit_seq 1\~4에는 buy_id가 1, hit_seq 5\~10에 buy_id가 2가 됩니다.

In [25]:
holist = ['2019-08-15','2019-09-12','2019-09-13'] # 공휴일 list

df1_online.sess_dt = pd.to_datetime(df1_online.sess_dt, format='%Y%m%d') # sess_dt col의 자료형 변경
df1_online['day'] = df1_online['sess_dt'].dt.weekday #요일숫자(0-월, 1-화) (=dayofweek)
df1_online['holiday'] = df1_online['day'].apply(lambda x: (x==5)|(x==6)).astype(np.int64)
df1_online['holiday'] = df1_online['holiday'] + df1_online['sess_dt'].apply(lambda x: x.strftime('%Y-%m-%d') in holist)
df1_online.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,cum_act_0,cum_act_1,cum_act_2,cum_act_3,cum_act_4,cum_act_5,cum_act_6,cum_act_7,day,holiday
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,1,0,0,0,0,0,0,0,2,0
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,2,0,0,0,0,0,0,0,2,0
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,3,0,0,0,0,0,0,0,2,0
3,1,2,1,0,A01,2019-09-22,14:09,41584,NaN,초등가을잠바,...,4,0,0,0,0,0,0,0,6,1
4,1,2,2,0,A01,2019-09-22,14:10,56113,NaN,초등가을점퍼,...,5,0,0,0,0,0,0,0,6,1


<br>

### 3. 변수 생성: `hour`
- `hit_tm`을 시간대별 분류를 통해 카테고리화하였습니다.
- (예) 23:35 -> 23, 23:47 -> 23

In [26]:
df1_online['hour'] = df1_online['hit_tm'].apply(lambda x: np.int8(x[:2]))
df1_online.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,cum_act_1,cum_act_2,cum_act_3,cum_act_4,cum_act_5,cum_act_6,cum_act_7,day,holiday,hour
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,0,0,0,0,0,0,0,2,0,16
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,0,0,0,0,0,0,0,2,0,16
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,0,0,0,0,0,0,0,2,0,16
3,1,2,1,0,A01,2019-09-22,14:09,41584,NaN,초등가을잠바,...,0,0,0,0,0,0,0,6,1,14
4,1,2,2,0,A01,2019-09-22,14:10,56113,NaN,초등가을점퍼,...,0,0,0,0,0,0,0,6,1,14


<br>

### 4. 변수 생성: `latest`
- 고객의 과거 행동을 담은 변수입니다. 
    - `latest_kwd_1`~`latest_kwd_6`: 가장 최근 검색한 여섯 개의 키워드를 담은 변수입니다.
    - `latest_pv_hr`
        - `pv_hr`은 온라인행동데이터의 `세션내총페이지뷰수/세션내총시간`으로 단위 페이지당 머무른 시간을 나타내는 서핑속도입니다.
        - `latest_pv_hr_1` \~ `latest_pv_hr_3`: 최근 쇼핑의 3개의 페이지 서핑 속도를 반영합니다.

In [27]:
sech_clac = pd.read_csv('./data/sech_clac_00.csv')

In [30]:
df1_online['sech_clac_nm2'] = sech_clac['sech_clac_nm2']
df1_online.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,cum_act_2,cum_act_3,cum_act_4,cum_act_5,cum_act_6,cum_act_7,day,holiday,hour,sech_clac_nm2
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,0,0,0,0,0,0,2,0,16,NaN
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,0,0,0,0,0,0,2,0,16,NaN
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,0,0,0,0,0,0,2,0,16,NaN
3,1,2,1,0,A01,2019-09-22,14:09,41584,NaN,초등가을잠바,...,0,0,0,0,0,0,6,1,14,Jewelry
4,1,2,2,0,A01,2019-09-22,14:10,56113,NaN,초등가을점퍼,...,0,0,0,0,0,0,6,1,14,Women's Shoes


In [32]:
test = df1_online[:50].copy()

In [33]:
test

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,cum_act_2,cum_act_3,cum_act_4,cum_act_5,cum_act_6,cum_act_7,day,holiday,hour,sech_clac_nm2
0,1,1,1,0,A01,2019-09-11,16:14,11880,NaN,과일선물세트,...,0,0,0,0,0,0,2,0,16,NaN
1,1,1,2,0,A01,2019-09-11,16:15,22432,NaN,과일선물세트 백화점,...,0,0,0,0,0,0,2,0,16,NaN
2,1,1,3,0,A01,2019-09-11,16:15,36140,NaN,과일바구니,...,0,0,0,0,0,0,2,0,16,NaN
3,1,2,1,0,A01,2019-09-22,14:09,41584,NaN,초등가을잠바,...,0,0,0,0,0,0,6,1,14,Jewelry
4,1,2,2,0,A01,2019-09-22,14:10,56113,NaN,초등가을점퍼,...,0,0,0,0,0,0,6,1,14,Women's Shoes
5,1,2,3,0,A01,2019-09-22,14:10,70025,NaN,초등남아옷,...,0,0,0,0,0,0,6,1,14,Women's Shoes
6,1,2,4,0,A01,2019-09-22,14:11,119527,NaN,초등남아점퍼,...,0,0,0,0,0,0,6,1,14,Women's Shoes
7,1,2,5,0,A01,2019-09-22,14:11,162327,NaN,노스페이스키즈,...,0,0,0,0,0,0,6,1,14,Men's Socks and Hosiery
8,1,2,6,0,A01,2019-09-22,14:12,178832,NaN,노스페이스초등,...,0,0,0,0,0,0,6,1,14,NaN
9,1,2,7,0,A01,2019-09-22,14:12,192367,NaN,노스페이스키즈,...,0,0,0,0,0,0,6,1,14,NaN


<br>

### 5. 변수 생성 : `prefer_dvc`
- 고객이 주로 사용하는 기기를 담은 변수입니다. 
- 상품 
- (예) 

In [57]:
df1_online.loc[df1_online['dvc_ctg_nm']=='unknown', 'dvc_ctg_nm'] = 'mobile_app'

In [67]:
test = df1_online.groupby('clnt_id')['dvc_ctg_nm'].apply(lambda x: ', '.join(set(x)))

In [69]:
test.value_counts()

mobile_web                38626
mobile_app                20994
PC                        12630
mobile_web, PC              111
mobile_app, mobile_web       24
mobile_app, PC               14
Name: dvc_ctg_nm, dtype: int64

<br>

### 6. 변수 생성: `trfc_src`
- 고객이 어떤 유입 채널을 선호하는가?
- 상품 
- (예) clnt_id, sess_id 가 모두 1인 고객의 hit_seq가 10까지 있는 경우
    - hit_seq가 4, 10에 구매가 발생했을 경우,
    - hit_seq 1\~4에는 buy_id가 1, hit_seq 5\~10에 buy_id가 2가 됩니다.

In [56]:
test = df1_online.groupby('clnt_id')['trfc_src'].apply(lambda x: ', '.join(set(x)))

In [72]:
df1_online['trfc_src'].value_counts()

DIRECT      1715541
unknown      826946
PUSH         523300
WEBSITE       49472
PORTAL_1      40669
PORTAL_2      30642
PORTAL_3       9792
Name: trfc_src, dtype: int64

<br>

### 7. 외부데이터: `기온`, `습도`, `강수형태`, `강수량`
- 기상청 데이터를 활용하여 7월-9월의 데이터를 추가하겠습니다. 
- 습도, 기온, 강수형태, 강수량 feature을 생성합니다. 

In [30]:
TEMP = pd.read_csv('./data/기온_201907_201909.csv')
HUM = pd.read_csv('./data/습도_201907_201909.csv')
PTY = pd.read_csv('./data/강수형태_201907_201909.csv')
R06 = pd.read_csv('./data/강수량_201907_201909.csv')

비구매 고객이든 구매고객이든 향후 어떤 제품을 선호할지 예측하고 추천하는 시스템. 
비구매 고객 -> Trade 데이터에 있는 feature 값들이 Nan으로. 


<br>

### 데이터 처리: `Trade + Item`
- 거래 ID와 Item을 `pd_c` 기준으로 합쳐줍니다. 


In [ ]:
# 거래정보(trans_id)를 적어도 하나 가지고있는(결측값이 아닌) 세션만을 가져오기 위한 사용자정의함수입니다.
def det_(df):
    nrows = df.shape[0]
    if df['trans_id'].isnull().value_counts().iloc[0] != nrows:
        return df

In [ ]:
# trans_id 가 있는 세션만 가져옴
df1_transO = df1_known.groupby(['clnt_id', 'sess_id']).apply(lambda x : det_(x))
df1_transO.reset_index(drop = True, inplace = True)
df1_transO.head()

In [ ]:
df2_df4 = df2trade.merge(df4item, how = 'left')
df2_df4.head()

<br>

### 데이터 처리: `Online + Trade`
- merge를 하기 전 trans_id의 중복을 제거하기 위함입니다. 


In [7]:
trans_list = list(online[~online['trans_id'].isnull()]['trans_id'].unique())
idx_list = [online.index[0]]
for i in trans_list:
    idx_list.append(online[online['trans_id'] == i].index.max())

In [9]:
online.loc[idx_list]

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
0,1,1,1,0,A01,20190911,16:14,11880,NaN,과일선물세트,14.0,124.0,unknown,mobile_app
46,2,1,37,6,A03,20190728,23:47,791905,62037.0,NaN,61.0,911.0,DIRECT,unknown
145,2,8,68,6,A03,20190731,21:25,3251794,64691.0,NaN,85.0,3256.0,DIRECT,unknown
167,2,10,16,6,A03,20190801,22:00,306944,65505.0,NaN,23.0,353.0,DIRECT,unknown
247,2,19,17,6,A03,20190830,22:00,2227390,88495.0,NaN,29.0,2253.0,DIRECT,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196182,72418,3,12,6,A02,20190919,12:52,2359650,104648.0,NaN,22.0,2360.0,unknown,PC
3196189,72419,1,5,6,A02,20190902,21:19,743203,90888.0,NaN,39.0,743.0,unknown,mobile_web
3196201,72423,1,1,6,A01,20190917,11:48,116283,102596.0,NaN,24.0,114.0,unknown,mobile_app
3196293,72424,2,52,6,A03,20190924,17:12,2410463,108506.0,NaN,82.0,2445.0,PORTAL_1,PC


In [53]:
online.loc[idx_list]['trans_id'].value_counts()

39950.0     1
70687.0     1
104016.0    1
60237.0     1
91095.0     1
           ..
38786.0     1
60553.0     1
63074.0     1
99132.0     1
81920.0     1
Name: trans_id, Length: 50997, dtype: int64

In [34]:
trade['trans_id'].value_counts()[104537.0]

1

In [48]:
trade

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
0,2,42449.0,1,A02,1015.0,20190704,15:34,46430,1
1,2,62037.0,1,A03,92.0,20190729,23:47,36000,20
2,2,64691.0,1,A03,186.0,20190731,21:25,3790,1
3,2,64691.0,2,A03,151.0,20190731,21:25,3990,1
4,2,64691.0,3,A03,351.0,20190731,21:25,4690,1
...,...,...,...,...,...,...,...,...,...
599956,72424,136596.0,38,B01,1580.0,20190727,11:22,2500,1
599957,72424,136596.0,39,B01,391.0,20190727,11:22,12980,1
599958,72424,136596.0,40,B01,964.0,20190727,11:22,2980,1
599959,72424,136596.0,41,B01,1571.0,20190727,11:22,1380,1


In [46]:
trade[trade['biz_unit'].apply(lambda x: x.startswith('A'))]['trans_id'].value_counts()

51589.0     100
102723.0     72
39972.0      44
91173.0      42
42701.0      42
           ... 
105276.0      1
47611.0       1
66596.0       1
60066.0       1
65536.0       1
Name: trans_id, Length: 44937, dtype: int64

In [3]:
temp = pd.merge(online, 
                trade[['trans_id', 'pd_c']], 
                on='trans_id', how='left')

df_merge = pd.merge(temp[(temp['pd_c'].notnull())], 
                    item[['pd_c', 'clac_nm0']], 
                    on='pd_c')